<a href="https://colab.research.google.com/github/Ganeshpy3/chatbot/blob/main/chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import keras 
from keras.layers import Dense
from keras.models import Sequential
import pandas as pd
import numpy as np
import json
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
file=open("/content/sample_data/intent.json")

In [3]:
data=json.load(file)

In [4]:
print(data["intends"])

[{'tags': 'welcome', 'patterns': ['hi', 'hello', 'hey', 'hi', 'hey there'], 'response': ['hello', 'hey whats up', 'hi']}, {'tags': 'thankyou', 'patterns': ['bye', 'see you', 'thanks bye'], 'response': ['bye take care', 'see you :)']}, {'tags': 'intro', 'patterns': ['what is your name', 'who are you', 'what are you'], 'response': ['iam chitti a chatbot', 'Artificial chatbot']}, {'tags': 'question', 'patterns': ['how are you', 'how you', 'are you fine'], 'response': ['iam fine', ':)fine,how are you ?']}, {'tags': 'found', 'patterns': ['who created you', 'when you born', 'how you created'], 'response': ['Iam an Artificial chatbot Founded by Ganesh']}]


In [9]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [10]:
ps=PorterStemmer()

In [20]:
allwords=[]
intent=[]
text=[]
for intend in data["intends"]:
  # print(intend["tags"])
  inte=intend["tags"]
  for p in intend["patterns"]:
    intent.append(inte)
    li=[]
    p=word_tokenize(p)
    for val in p:
       valu=ps.stem(val.lower())
       li.append(valu)
       allwords.append(valu)
    text.append(" ".join(li))

In [21]:
intent

['welcome',
 'welcome',
 'welcome',
 'welcome',
 'welcome',
 'thankyou',
 'thankyou',
 'thankyou',
 'intro',
 'intro',
 'intro',
 'question',
 'question',
 'question',
 'found',
 'found',
 'found']

In [22]:
df=pd.DataFrame({"Text":text,"Intent":intent})

In [23]:
le=LabelEncoder()

In [24]:
Y=le.fit_transform(df["Intent"])

In [25]:
Y

array([4, 4, 4, 4, 4, 3, 3, 3, 1, 1, 1, 2, 2, 2, 0, 0, 0])

In [17]:
CV=CountVectorizer()

In [38]:
allwords=set(allwords)

In [55]:
def bag_of_words(words,allwords):
  words=word_tokenize(words)

  data=np.zeros(len(allwords))
  for idx,value in enumerate(allwords):
    if value in words:
      data[idx]=1

  return data




In [58]:
Xbag=[]
for sentence in df["Text"]:
  Xbag.append(bag_of_words(sentence,allwords))



In [62]:
X=pd.DataFrame(Xbag,columns=allwords)

In [63]:
X.shape

(17, 19)

In [65]:
from sklearn.svm import LinearSVC

In [70]:
model=Sequential()
model.add(Dense(units=1000,activation="relu",kernel_initializer='he_uniform',input_dim=19))
model.add(Dense(units=50,activation="relu",kernel_initializer="he_uniform"))
model.add(Dense(units=5,activation="softmax"))

In [71]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 1000)              20000     
_________________________________________________________________
dense_7 (Dense)              (None, 50)                50050     
_________________________________________________________________
dense_8 (Dense)              (None, 5)                 255       
Total params: 70,305
Trainable params: 70,305
Non-trainable params: 0
_________________________________________________________________


In [72]:
model.compile(loss="sparse_categorical_crossentropy",metrics=["accuracy"])

In [73]:
classifier=model.fit(X,Y,epochs=20)

Epoch 1/20
1/1 [==============================] - 16s 16s/step - loss: 1.6309 - accuracy: 0.1765
Epoch 2/20
1/1 [==============================] - 0s 5ms/step - loss: 0.9118 - accuracy: 0.8235
Epoch 3/20
1/1 [==============================] - 0s 7ms/step - loss: 0.6282 - accuracy: 0.8824
Epoch 4/20
1/1 [==============================] - 0s 7ms/step - loss: 0.4136 - accuracy: 1.0000
Epoch 5/20
1/1 [==============================] - 0s 8ms/step - loss: 0.2836 - accuracy: 1.0000
Epoch 6/20
1/1 [==============================] - 0s 7ms/step - loss: 0.2099 - accuracy: 1.0000
Epoch 7/20
1/1 [==============================] - 0s 6ms/step - loss: 0.1649 - accuracy: 1.0000
Epoch 8/20
1/1 [==============================] - 0s 5ms/step - loss: 0.1346 - accuracy: 1.0000
Epoch 9/20
1/1 [==============================] - 0s 8ms/step - loss: 0.1108 - accuracy: 1.0000
Epoch 10/20
1/1 [==============================] - 0s 8ms/step - loss: 0.0945 - accuracy: 1.0000
Epoch 11/20
1/1 [=====================

In [77]:
a=bag_of_words("who are you",allwords)

In [87]:
b=pd.DataFrame([a])

In [89]:
np.argmax(model.predict(b))

1